# p300 Speller Application
By: Thomas Blalock<br>
Date: December 6, 2023<br>
### Abstract
[INSERT ABSTRACT]
### Contents
- Introduction
- Hardware Setup
- Hardware-Software Integration
- Stimulus-Producing Software
- Data Collection
- Data Preprocessing
- Data Loading
- Model Development
- Model Training
- Results
- Next Steps
- References

### Introduction
A brain-computer interface (BCI) reads and interprets the electromagnetic fields created by neural activity. This project employs a specific type of non-invasive BCI called electroencephalograpy (EEG), which reads electrical signals using electrodes that sit on a person's scalp. this project seeks to create a system called a speller application to enable people to communicate without any motor movement. Different BCI spellers utilize different signals in the brain. These various strategies for speller applications are referred to as speller paradigms. We employ the p300 speller paradigm, which utilizes the p300 novelty response in the brain that can be triggered from flashing colors as well as a digital keyboard that flashes in different patterns to determine what a subject is looking at. On the back-end, a classification model parses the data to determine if the p300 response is present. Significant noise is introduced in EEG systems, originating from other processes in the brain, the environment, and other processes in the body, such as blinking. Every part of this system pipeline can be optimized. The quality of the stimulus-producing software determines the size of the p300 response. The quality of data cleaning and feature extraction could determine if the model converges. And lastly, the quality of the machine learning model determines if the system yields the correct response. A literature report linked [here](https://docs.google.com/document/d/1aNbAXmB_J7fFUMmZ_Ofe9Z-wUfX3AwkIEsbiiaiTxu8/edit?usp=sharing) further details the methods and paradigms mentioned in this report.

### Hardware Setup
In this project, we used a 3D-printed plastic frame with eight OpenBCI electrodes and an OpenBCI Cyton Board. The equipment was provided by Dr. Mello from the USAFA ACCR. The electrodes were placed in the p300 configuration found at this [link](https://www.researchgate.net/figure/Common-electrode-setup-for-P300-spellers-according-to-8-Eight-EEG-electrodes-are_fig1_221583051#:~:text=Eight%20EEG%20electrodes%20are%20placed,is%20attached%20to%20the%20rig). The wires were oriented orthogonal to each other to minimize cross-contamination.<br><br>
![Image of the Fully Contructed EEG Helmet](content/dummy_img.png)

### Hardware-Software Integration
The software generated to interact with the helmet assumes that the helmet transmits the signal to the computer through a USB port. Since only one process can access a USB port at a time, our scripts and the OpenBCI GUI cannot be running at the same time. A 'Board' object was created to handle all interactions with the EEG helmet. The code is modular enough to that it would be very easy to change the script to receive the signal over wifi rather than the USB port. Run the following code block to define this object.

In [1]:
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.board_shim import BrainFlowInputParams
import time


class Board:
    """
    This class is a wrapper for the BrainFlow Cyton board. It allows you to
    start and stop the stream, and get data from the board.
    """

    def __init__(self, port = 'COM4'):
        """Initializes the board and prepares the session.
        
        Args:
            port (str): The port the board is connected to.
        """
        self.port = port
        # Prep board / data stream
        BoardShim.enable_dev_board_logger()
        params = BrainFlowInputParams()
        params.serial_port = self.port
        self.board = BoardShim(BoardIds.CYTON_BOARD, params)
        self.board.prepare_session()
        

    def __del__(self):
        """Releases the session."""
        self.board.release_session()    
        time.sleep(1.5) # It crashes if you  restart the board too quickly

    def start_stream(self):
        """Starts the stream."""
        self.board.start_stream()

    def stop_stream(self):
        """Stops the stream."""
        self.board.stop_stream()

    def get_data(self):
        """Gets the data from the board.
        
        Returns:
            list: The data from the board.
        """
        return self.board.get_board_data()

### Stimulus-Producing Software
The end goal is to have a QWERTY keyboard that utilizes a binary search with the p300 responses to find the letter that a subject is looking at. It was determined that the machine learning model would converge more quickly if it was first trained on a more prominent p300 signal and then trained on the QWERTY keyboard. There two GUI objects that flash different colors to stimulate the P300 response. The first version flashes different-sized boxes while the second version performs the binary search using a QWERTY keyboard with some non-letter values. 

The first GUI is just a flashing box. It is designed to elicit a strong p300. The data collected using this GUI will be used to pretrain the model. The model will theoretically converge faster on a dataset with a more pronounced loss landscape. The GUI enables the user to specify differently sized and shaped box. This variation allows the user to vary the size of the p300 response since the size of the flashing box will likely correlate to the size of the p300 response. Multiples colors flash during a single sample. The colors and time between flashes are randomized to maximize the p300 response. The user controls the GUI by pressing any button to start a session and pressing any button to end the session. The following code block defines the 'Box_GUI' class.<br>
<video width=" " height=" " 
       src="content/dummy_video.mp4"  
       controls>
</video>

In [2]:
import pygame
import random
import pyautogui


class Box_GUI:
    """
    This class is a wrapper for the pygame GUI. It allows you to
    flash a box on the screen and wait for a button press.
    """

    # Constants
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    RED = (255, 0 , 0)
    GREEN = (0, 255, 0)
    YELLOW = (255, 255, 0)
    PURPLE = (255, 0, 255)
    BLUE = (0, 0, 255)
    ORANGE = (255, 165, 0)
    PINK = (255, 192, 203)
    COLOR_LIST = [RED, GREEN, YELLOW, PURPLE, BLUE, ORANGE, PINK]

    def __init__(self, window_size = None):
        """Initializes the pygame GUI and determines the window size.
        
        args:
            window_size (tuple): The size of the window. If None, the size of the screen is used.
        """
        if window_size == None:
            width, height = pyautogui.size()
        else:
            width, height = window_size
        self.window_size = (width, height-(height/20))

        # Prep pygame window
        pygame.init()
        self.screen = pygame.display.set_mode(self.window_size)
        self.screen.fill(self.BLACK)

    def __del__(self):
        pygame.quit()
    
    def reset_screen(self, box_size):
        """Resets the screen to black and draws a box in the center.
        
        args:
            box_size (tuple): The size of the box.
        """
        box_size = self.box_size_parser(box_size)
        center = ((self.window_size[0] - box_size[0])/2, (self.window_size[1] - box_size[1])/2)
        pygame.draw.rect(self.screen, self.GREEN, (center[0], center[1], box_size[0], box_size[1]), 0)
        pygame.display.update()
        pygame.display.flip()

    def flash_box(self, box_size, color='random'):
        """Flashes a box on the screen.
        
        args:
            box_size (tuple or string or int):
                tuple: (x_width, y_width) of the box
                string: 'screen' or 'full' to fill the screen
                int: the height and width of the box
            color (str): The color of the box. If 'random', a random color is chosen.
        """
        if color == 'random':
            color = random.choice(self.COLOR_LIST)
        box_size = self.box_size_parser(box_size)
        center = ((self.window_size[0] - box_size[0])/2, (self.window_size[1] - box_size[1])/2)
        pygame.draw.rect(self.screen, color, (center[0], center[1], box_size[0], box_size[1]), 0)
        pygame.display.update()
        pygame.display.flip()

    def button_press(self):
        """Listener for a button press."""
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                return True
        return False
    
    def box_size_parser(self, box_size):
        """Parses the box size input and returns.
        
        args:
            box_size (tuple or string or int):
                tuple: (x_width, y_width) of the box
                string: 'screen' or 'full' to fill the screen
                int: the height and width of the box
        
        returns:
            tuple: (x_width, y_width) of the box
        """
        if  type(box_size) == str:
            if box_size == 'screen' or box_size == 'full':
                return self.window_size
        elif type(box_size) == int:
            return (box_size, box_size)
        elif type(box_size) == tuple and len(box_size) == 2:
            return box_size
        else:
            raise ValueError('Invalid box size input: ' + type(box_size) + '. Must be string, int, or 2-valued tuple.')

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


The other GUI is a QWERTY keyboard that performs a binary search to locate the letter a subject is looking at. Multiples colors flash during a single sample. The colors and time between flashes are randomized to maximize the p300 response. The user controls the GUI by pressing any button to start a session and pressing any button to end the session. The following code block defines the 'Keyboard_GUI' class.<br>
<video width=" " height=" " 
       src="content/dummy_video.mp4"  
       controls>
</video>

In [ ]:
class Keyboard_GUI:

    # Constants
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    RED = (255, 0 , 0)
    GREEN = (0, 255, 0)
    YELLOW = (255, 255, 0)
    PURPLE = (255, 0, 255)
    BLUE = (0, 0, 255)
    ORANGE = (255, 165, 0)
    PINK = (255, 192, 203)
    COLOR_LIST = [RED, YELLOW, PURPLE, BLUE, ORANGE, PINK]
    KEY_COLOR_LIST = [GREEN, YELLOW, PINK, WHITE]

    def __init__(self, window_size=None):
        if window_size == None:
            width, height = pyautogui.size()
        else:
            width, height = window_size
        self.window_size = (width, height-(height/20))
        self.window_size = (width, height-(height/20))
        self.keyboard = [["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "Backspace", "End"],
                    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P","(", ")"],
                    ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";", "'"], 
                    ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/", "Enter"],
                    ["       "]]

        # Prep pygame window
        pygame.init()
        self.screen = pygame.display.set_mode(self.window_size)
        self.screen.fill(self.BLACK)
        self.font = pygame.font.SysFont("Arial", 20)

    def __del__(self):
        pygame.quit()
    
    def reset_screen(self, button_size = None, flash_keys = [], color='random', target_letter=None):
        if color == 'random':
            flash_color = random.choice(self.COLOR_LIST)
            flash_color_key = random.choice(self.KEY_COLOR_LIST)
        else:
            flash_color = color
        col_margin = 5
        row_margin = 5
        if button_size == None:
            button_size = (self.window_size[0]/13, self.window_size[1]/5)

        # Draw a big black box over everything
        pygame.draw.rect(self.screen, self.BLACK, (0, 0, self.window_size[0], self.window_size[1]), 0)
        
        # Draw buttons
        for y, row in enumerate(self.keyboard):
            for x, key in enumerate(row):

                if key in flash_keys:
                    color = flash_color
                    key_color = flash_color_key
                else:
                    color = self.GREEN
                    key_color = self.BLACK

                if y>=1 and y<=3: # Letters
                    indent = 20
                elif key=="       ": # Space bar
                    button_size = (button_size[0]*5, button_size[1])
                    indent = int(self.window_size[0]/2 - button_size[0]/2)
                else:
                    indent = 0
                pygame.draw.rect(self.screen, color, (indent+x*(button_size[0] + col_margin)+col_margin, 
                                                           y*(button_size[1] + row_margin)+row_margin, button_size[0], button_size[1]))
                text_surface = self.font.render(key, True, key_color)
                self.screen.blit(text_surface, (indent+x*(button_size[0] + col_margin)+col_margin+button_size[0]/2 - text_surface.get_width()/2, 
                                                y*(button_size[1] + row_margin)+row_margin+button_size[1]/2 - text_surface.get_height()/2))
        
        # Draw target letter on bottom left corner
        if target_letter is not None:
            text_surface = self.font.render(target_letter, True, self.WHITE)
            self.screen.blit(text_surface, (5 + text_surface.get_width(), 
                                            self.window_size[1] - 5 - text_surface.get_height()))

        pygame.display.flip()

    def button_press(self):
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                return True
        return False
    
    def get_keyboard(self):
        return self.keyboard
    
    def flash_keys(self, keys, color='random', target_letter=None):
        if type(keys) == str:
            new_keys = [keys]
        elif type(keys) == list:
            new_keys = []
            if type(keys[0]) == list:
                for l in keys:
                    new_keys.extend(l)
            else:
                new_keys = keys
        else:
            raise ValueError('Invalid key input: ' + type(keys) + '. Must be string or list of strings.')
        
        self.reset_screen(flash_keys = new_keys, color=color, target_letter=target_letter)

    def get_search_pattern(self, prev_flash=None, response=True):
        if prev_flash is None:
            pattern = self.nested_to_1d_list(self.keyboard[:int(len(self.keyboard)/2)])
            full = self.nested_to_1d_list(self.keyboard)
        else:
            prev_flash = self.nested_to_1d_list(prev_flash)
            if response:
                pattern = prev_flash[:int(len(prev_flash)/4)]
                full = prev_flash[:int(len(prev_flash)/2)]
            else:
                pattern = prev_flash[int(len(prev_flash)/2):int(3*len(prev_flash)/4)]
                full = prev_flash[int(len(prev_flash)/2):]
        
        return pattern, full
    
    def nested_to_1d_list(self, l):
        new_l = []
        if type(l)!=list:
            new_l.append(l)
        else:
            for item in l:
                new_l.extend(self.nested_to_1d_list(item))
        return new_l

### Data Collection


### Data Preprocessing


### Data Loading


### Model Development


### Model Training


### Results


### Next Steps


### References